In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img


In [2]:
def get_image_arrays(image_id, image_path):
    X = []
    image = load_img(image_path + image_id, target_size=(224, 224))
    image_array = img_to_array(image)

    X.append(image_array)

    X_array = np.asarray(X, dtype='float32')
    X_array /= 255.

    return X_array


def get_image_predictions(image_array, model_path):
    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test the model on random input data.
    input_shape = input_details[0]['shape']
    input_data = image_array
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data


def show_image(image_id, image_path):
    image_id_dict = dict(image_id).values()
    image_id_string = list(image_id_dict)[0]
    img = mpimg.imread(image_path + image_id_string)
    plt.imshow(img, interpolation='nearest', aspect='auto')
    plt.show()


In [3]:

from sklearn.metrics import accuracy_score, roc_auc_score
import pandas as pd
import numpy as np
import pickle


In [4]:
data_dir = r'E:\datasets\MADE\3_graduation\parthplc\archive\data\\'

train_path = data_dir + 'train.jsonl'
dev_path = data_dir + 'dev.jsonl'


train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [5]:
test_data.head(3)

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...


In [6]:
TFLITE_FILE_PATH = 'image_model.tflite'

# TFIDF Model
model = 'tfidf_model.pickle'
vectorizer = 'tfidf_vectorizer.pickle'
tfidf_model = pickle.load(open(model, 'rb'))
tfidf_vectorizer = pickle.load(open(vectorizer, 'rb'))

C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Temp\ipykernel_1388\3537662425.py:7: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_vectorizer = pickle.load(open(vectorizer, 'rb'))
C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.1 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitatio

In [7]:
for row in test_data.values:
    print(row)
    break

[8291 'img/08291.png' 1 'white people is this a shooting range']


In [8]:
predict = []
y_true = []

for row in test_data.values:
    label = row[2]
    image_id = row[1]
    text = [row[3]]

    image_array = get_image_arrays(image_id, data_dir)
    image_prediction = get_image_predictions(image_array, TFLITE_FILE_PATH)
    y_pred_image = np.argmax(image_prediction, axis=1)
    #print('Image Prediction Probabilities:')
    #print(image_prediction)


    transformed_text = tfidf_vectorizer.transform(text)
    text_prediction = tfidf_model.predict_proba(transformed_text)
    y_pred_text = np.argmax(text_prediction, axis=1)
    #print('Text Prediction Probabilities:')
    #print(text_prediction)


    # Ensemble Probabilities
    ensemble_prediction = np.mean(np.array([image_prediction, text_prediction]), axis=0)
    y_pred_ensemble = np.argmax(ensemble_prediction, axis=1)
    #print(ensemble_prediction)


    predicted_label = y_pred_ensemble[0]
    y_true.append(label)
    predict.append({"label":predicted_label, "proba":np.max(ensemble_prediction, axis=1)[0]})
    #print(predict[-1])
    #break

In [9]:
[p["label"] for p in predict][:3]

[1, 0, 0]

In [10]:
[p["proba"] for p in predict][:3]

[0.5320135628288469, 0.7226082252364714, 0.5670788375148931]

In [11]:
acc_score = accuracy_score(y_true, [p["label"] for p in predict])
auc_score = roc_auc_score(y_true, [p["proba"] for p in predict])
acc_score, auc_score

(0.53, 0.44279999999999997)

In [12]:
predict = []
y_true = []

for row in train_data.values:
    label = row[2]
    image_id = row[1]
    text = [row[3]]

    image_array = get_image_arrays(image_id, data_dir)
    image_prediction = get_image_predictions(image_array, TFLITE_FILE_PATH)
    y_pred_image = np.argmax(image_prediction, axis=1)
    #print('Image Prediction Probabilities:')
    #print(image_prediction)


    transformed_text = tfidf_vectorizer.transform(text)
    text_prediction = tfidf_model.predict_proba(transformed_text)
    y_pred_text = np.argmax(text_prediction, axis=1)
    #print('Text Prediction Probabilities:')
    #print(text_prediction)


    # Ensemble Probabilities
    ensemble_prediction = np.mean(np.array([image_prediction, text_prediction]), axis=0)
    y_pred_ensemble = np.argmax(ensemble_prediction, axis=1)
    #print(ensemble_prediction)


    predicted_label = y_pred_ensemble[0]
    y_true.append(label)
    predict.append({"label":predicted_label, "proba":np.max(ensemble_prediction, axis=1)[0]})
    #print(predict[-1])
    #break
    
acc_score = accuracy_score(y_true, [p["label"] for p in predict])
auc_score = roc_auc_score(y_true, [p["proba"] for p in predict])
acc_score, auc_score

(0.7185882352941176, 0.2579691983756956)

In [13]:
demo_data = pd.read_csv('demo_data.csv')

predict = []
y_true = []

for row in demo_data.values:
    print(row)
    break
    
    

['48539.png'
 "the woman you met who admits she's wrong, apologizes and changes her ways"
 0]


In [14]:
demo_data = pd.read_csv('demo_data.csv')

predict = []
y_true = []

for row in demo_data.values:
    label = row[2]
    image_id = 'img/' + row[0]
    text = [row[1]]

    image_array = get_image_arrays(image_id, data_dir)
    image_prediction = get_image_predictions(image_array, TFLITE_FILE_PATH)
    y_pred_image = np.argmax(image_prediction, axis=1)
    #print('Image Prediction Probabilities:')
    #print(image_prediction)


    transformed_text = tfidf_vectorizer.transform(text)
    text_prediction = tfidf_model.predict_proba(transformed_text)
    y_pred_text = np.argmax(text_prediction, axis=1)
    #print('Text Prediction Probabilities:')
    #print(text_prediction)


    # Ensemble Probabilities
    ensemble_prediction = np.mean(np.array([image_prediction, text_prediction]), axis=0)
    y_pred_ensemble = np.argmax(ensemble_prediction, axis=1)
    #print(ensemble_prediction)


    predicted_label = y_pred_ensemble[0]
    y_true.append(label)
    predict.append({"label":predicted_label, "proba":np.max(ensemble_prediction, axis=1)[0]})
    #print(predict[-1])
    #break
    
acc_score = accuracy_score(y_true, [p["label"] for p in predict])
auc_score = roc_auc_score(y_true, [p["proba"] for p in predict])
acc_score, auc_score

(0.64, 0.3549999999999999)